# Foreacasting - Metric Per Entity

#### Version 1.0

### TODO

- Read aggreated data
- Forecast 30 years for metric per entity

### Observations

## Setup

### Import Packages

In [53]:
from datetime import datetime, date
from pprint import pprint
from collections import OrderedDict
import os

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
# import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

from prophet import Prophet

# Set ipython's max row display
# pd.set_option('display.max_row', 1000)
# Set iPython's max column width
pd.set_option('display.max_columns', 50)

# sns.set_style("darkgrid")
plt.rcParams['figure.figsize'] = [20, 10]

import warnings
warnings.filterwarnings('ignore')
import logging
logger = logging.getLogger('cmdstanpy')
logger.addHandler(logging.NullHandler())
logger.propagate = False
logger.setLevel(logging.CRITICAL)

### Functions

In [54]:
def masked_heatmap():
    mask = np.zeros_like(corr)
    mask[np.triu_indices_from(mask)] = True
    fig,ax = plt.subplots()
    sns.heatmap(corr,center = 0, fmt = ".3f", square = True, annot= True, mask=mask)

## Global Variables

#### Dataset Parameters

In [55]:
file_name = '../data/silver_tables/yearly_values_per_entity.csv'

year_range = [1960, 2022]
entity_category = 'Country'

forecast_horizon = 30

In [56]:
index_columns = [
    'Entity',
    'Code',
    'Year',
]
forecase_columns = [
    'Annual CO2 emissions',
    #'Annual change in primary energy consumption (%)',
    # 'Per capita electricity (kWh)',
    'Fossil fuels per capita (kWh)',
    # 'Fossil fuels (% equivalent primary energy)',
    # 'Renewables per capita (kWh - equivalent)', 'Renewables (% electricity)',
    # 'Solar Generation - TWh', 'prod of Electricity from solar (TWh)',
]

## Data Prep

In [64]:
df_original = pd.read_csv(file_name)

In [66]:
# df_original = df_original.loc[
#     (df_original['Year'] > year_range[0])
#     & (df_original['Year'] < year_range[1])
#     & (df_original['Entity_Category'] == entity_category)
# ][[*index_columns,*forecase_columns]]

df_original = df_original.loc[
    (df_original['Year'] > year_range[0])
    & (df_original['Year'] < year_range[1])
][[*index_columns,*forecase_columns]]

In [67]:
df_original.head(2)

,Entity,Code,Year,Annual CO2 emissions,Fossil fuels per capita (kWh)
34,Aruba,ABW,1961,645553.0,NaN
35,Aruba,ABW,1962,708942.0,NaN


In [68]:
df_original['Year'] = pd.to_datetime(df_original['Year'], format='%Y')

### Create Per Country Dataframes

In [69]:
x = 'Year'
y = 'Annual CO2 emissions'

In [70]:
entity_dfs_dict = {}

for entity in df_original['Entity'].unique():
    entity_dfs_dict[entity] = df_original.loc[df_original['Entity'] == entity][[x, y]]
    entity_dfs_dict[entity].columns = ['ds', 'y']

In [71]:
entity_dfs_dict[entity].head()

,ds,y
21818,1961-01-01,5061263.0
21819,1962-01-01,4888959.0
21820,1963-01-01,4700377.0
21821,1964-01-01,4469069.0
21822,1965-01-01,5209198.0


## Forecast Generation 

In [72]:
def create_predict_df_for_entity(entity_name):
    df = entity_dfs_dict[entity_name]
    
    m = Prophet(
        yearly_seasonality='auto',
        weekly_seasonality=False,
        daily_seasonality=False,
    )
    m.fit(df)
    
    future = m.make_future_dataframe(periods=forecast_horizon, freq='Y')
    forecast = m.predict(future)
    
    df = df.set_index('ds').join(forecast.set_index('ds')).reset_index()
    df['Entity'] = entity_name
    
    return df

In [73]:
def create_pred_df(entity_dfs_dict):
    all_pred_df = pd.DataFrame()

    try:
        for entity_name in entity_dfs_dict:
            print(f"Forecasting for {entity_name} done")
            
            try:
                df = create_predict_df_for_entity(entity_name)    
            except ValueError:
                print('Not enough observations for forecasting')
                df = pd.DataFrame()

            all_pred_df = pd.concat([all_pred_df, df])
        return all_pred_df
    
    except KeyboardInterrupt:
        return all_pred_df

In [ ]:
all_entities_pred_df = create_pred_df(entity_dfs_dict)

Forecasting for Aruba done
Forecasting for Afghanistan done
Forecasting for Angola done
Forecasting for Anguilla done
Forecasting for Albania done
Forecasting for Andorra done
Forecasting for Netherlands Antilles done
Not enough observations for forecasting
Forecasting for United Arab Emirates done
Forecasting for Argentina done
Forecasting for Armenia done
Forecasting for American Samoa done
Not enough observations for forecasting
Forecasting for Antarctica done
Forecasting for Antigua And Barbuda done
Forecasting for Australia done
Forecasting for Austria done
Forecasting for Azerbaijan done
Forecasting for Burundi done
Forecasting for Belgium done
Forecasting for Benin done
Forecasting for Bonaire Sint Eustatius And Saba done
Forecasting for Burkina Faso done
Forecasting for Bangladesh done
Forecasting for Bulgaria done
Forecasting for Bahrain done
Forecasting for Bahamas done
Forecasting for Bosnia And Herzegovina done
Forecasting for Belarus done
Forecasting for Belize done
Foreca

## Save Tables

In [ ]:
all_entities_pred_df = all_entities_pred_df.rename(columns={'ds':'Year', 'y': f'{y}', 'yhat': f'{y}_predicted'})

In [ ]:
all_entities_pred_df.head()

In [ ]:
all_entities_pred_df['Entity'].value_counts()

In [ ]:
all_entities_pred_df.to_csv('../data/forecasts_tables/all_entities_forecasts.csv', index=False)